In [17]:
#Essa parte vai ler outros arquivos do tipo CSV e vai converter para Sqlite3 e SQL.
# Isso vai facilitar a inserção de grandes volumes dados no banco de dados.
# A gente vai começar o desenvolvimento disso no Jupyter Notbook porque é mais fácil e rápido de fazer testes. Depois é só copiar pra cá que tá tudo certo!
# A estrutura de leitura vai ser a mesma que já desenvolvi pra ler planilhas porque ela é muito funcional!
# A estrutura de escrita no outro formato é que vou ter que adaptar pra usar os itens lidos no documento 'csv'

import datetime as dt
from openpyxl import Workbook, load_workbook
from openpyxl.styles import PatternFill, Font
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)#Isso aqui é pra tirar um aviso de erro

In [18]:
tempoInicial = dt.datetime.now().minute
print(f'Minutagem inicial {tempoInicial}')

#Criando lista de diretórios de arquivos do excel
arquivos = glob.glob('01 - MÊS ATUAL\*CONTROLE DIÁRIO*\*.xlsm') #Aqui ele vai pegar qualquer item com a extensão 'xlsm' do diretório passado
print("\n-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=LISTA DE ARQUIVOS NA PASTA=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-\n")
c = 0
for i in arquivos:
    print(arquivos[c])
    c += 1

Minutagem inicial 29

-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=LISTA DE ARQUIVOS NA PASTA=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-



In [19]:
#Listando as abas "ATIVOS" lidas de dentro dos documentos do excel listados em "arquivos"
#Obs: Até aqui não dá pra ler diretamente da variável "arquivos" porque ela só contém os nomes dos arquivos!
print("\n-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=INICIANDO A LEITURA DOS ARQUIVOS=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-\n")

mesAnterior = [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
dataMes = dt.datetime.now().month #Posição da coluna do mês

if dataMes == 1:
    dataMes = mesAnterior[11]
else:
    dataMes = mesAnterior[dataMes - 2]
print(f'Aqui estamos verificando o mês da coluna {dataMes}\n')

abas_ATIVOS = [] #Lista de abas com o nome "ATIVOS". cada elemento dentro da lista vai ser um DataFrame da aba "ATIVOS" de cada arquivo
c = 0
for i in arquivos:
    abas_ATIVOS.append(pd.read_excel(arquivos[c], sheet_name = 'ATIVOS', usecols=[2, 4, dataMes, 17, 18, 21, 22, 23, 25, 26,  28, 29, 30, 31, 70, 94, 95, 97])) #Adicionando as abas "ATIVOS" dentro da lista "abas_ATIVOS"
    #acho que dá pra fazer a verificação que de colunas aqui mesmo, que aí ele lê o arquivo, analisa e entrega logo os erros de uma por vez!
    print(f'Arquivo {c}: "{arquivos[c]}" lido com sucesso!')
    c +=1

print(f'\nForam lidos {len(arquivos)} arquivos!')
print(f'Foram encontradas: {len(abas_ATIVOS)} abas de "ATIVOS"!')


-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=INICIANDO A LEITURA DOS ARQUIVOS=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-

Aqui estamos verificando o mês da coluna 9


Foram lidos 0 arquivos!
Foram encontradas: 0 abas de "ATIVOS"!


In [12]:
print("\n-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-= DROPANDO COLUNAS DESNECESSÁRIAS E RENOMEANDO AS COLUNAS CORRETAMENTE =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-\n")
#Aqui a gente ajeita as tabelas, dropa colunas desnecessárias, linhas "NaN" e renomea as colunas de cada arquivo.
arqsplitsss = []
c = 0
for i in arquivos:
    #Dropando as colunas pelo índice! (Se for mexer aqui lembra que tem que usar o índice da coluna, não o nome, e mais o "axis=1", indicando que estamos mexendo com as colunas pelo índice)
    #IMPORTANTE: AQUI DA COLUNA 5 À 16 TODAS TEM QUE SER DROPADAS, MENOS A DO MÊS VIGENTE!
#    abas_ATIVOS[c] = abas_ATIVOS[c].drop(abas_ATIVOS[c].columns[colunasadropar], axis=1)
    
    #Dropando linhas "NaN", com base na coluna 0.
    abas_ATIVOS[c] = abas_ATIVOS[c].dropna(subset=[abas_ATIVOS[c].columns[0]])
    
    #Dropando a linha '0' porque ela tem só títulos e tá tudo fora de padrão, daí não servem
    abas_ATIVOS[c] = abas_ATIVOS[c].drop(0, axis=0)
    
    #Modificando o cabeçalhos das coluna de cada arquivo
    abas_ATIVOS[c].columns = ["N° FORM", "STATUS", "MÊS", "CONDIÇÃO SOCIAL", "NOME", "NASCIMENTO", "IDADE", "SEXO", "NOME DO PAI", "NOME DA MÃE", "CPF", "NIS", "RG OU RNE", "ÓRGÃO EMISSOR", "REGIÃO ADMINISTRATIVA", "DATA 1° ABORDAGEM", "DATA DA ÚLTIMA ABORDAGEM", "STATUS SEAS"]
    
    #Inserindo a coluna "EQUIPE" com o nome da equipe:
    arqsplits = arquivos[c].split(' - ') #Isso aqui é só pra pegar o número da equipe
    abas_ATIVOS[c].insert(0, 'EQUIPE', arqsplits[3]) #ISSO AQUI A GENTE VAI TENTAR PEGAR FAZENDO UM SPLIT PELO TRAÇO: '-' E PEGANDO UM BLOCO, É BOM PORQUE AÍ CONSIGO PEGAR O NOME DA CIDADE, DA EQUIPE E DO SUPERVISOR!
    c += 1
    
    #Isso aqui é pra adicionar o nome dos arquivos numa lista, pra depois puxar os mesmos nomes pras colunas que vão usar elas
    arqsplitsss.append(arqsplits)
#Total de abas "ATIVOS" encontradas
print(f'COLUNAS DROPADAS E RENOMEADAS de todas as {len(abas_ATIVOS)} abas de "ATIVOS"!')



-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-= DROPANDO COLUNAS DESNECESSÁRIAS E RENOMEANDO AS COLUNAS CORRETAMENTE =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-

COLUNAS DROPADAS E RENOMEADAS de todas as 0 abas de "ATIVOS"!


In [13]:
#Só teste aqui pra ver os arquivos que tão indo pra lista de arqsplitsss
'''for i in arqsplitsss:
    print(i)

#print(arqsplits[3], arqsplits[4])'''

'for i in arqsplitsss:\n    print(i)\n\n#print(arqsplits[3], arqsplits[4])'

In [14]:
print("\n-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-= ENCONTRANDO ERROS NAS PLANILHAS =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-\n")
#PRA CADA ARQUIVO TEM QUE PEGAR A COLUNA E PRA CADA LINHA TEM QUE PEGAR UM ITEM E FAZER UMA COMPARAÇÃO
c = 0
for i in arquivos:
    #NOME DO DOCUMENTO QUE ESTÁ SENDO ANALIZADO, SÓ PRECISA ESTAR NESSA PARTE PORQUE SE COLOCAR MAIS ADIANTE ELE DUPLICA O NOME NA EXIBIÇÃO.
    print('\033[1m' + f'{arquivos[c]}' + '\033[0m')

    #ACHANDO ERROS NA COLUNA 'STATUS'
    coluna = ['ATENDIDO', 'ACOMPANHADO', 'IDENTIFICADO']
    erroCol = []
    erro = []
    itens = 0
    erroNum = 0

    for i in abas_ATIVOS[c]['STATUS']: #Para cada item da coluna 'STATUS':
        if i not in coluna: #SE O ITEM NÃO ESTIVER NA LISTA 'COLUNA':
            erroCol.append(i) #ADICIONA O ITEM À LISTA DE ERROS
            erro.append(itens) #ADICIONA O NÚMERO DA LINHA QUE O ITEM ESTÁ NA LISTA 'ERRO'
        itens += 1

    if len(erroCol) != 0:
        print(f'FORAM ENCONTRADOS {len(erroCol)} ERROS NA COLUNA "STATUS":')
        for i in erro:
            print('\033[91m', '\033[1m', '\033[4m', f'LINHA {i+3} >>> ERRO: {erroCol[erroNum]}', '\033[0m', '\033[0m', '\033[0m')
            erroNum += 1
            
            
            
    #ACHANDO ERROS NA COLUNA 'MÊS'
    coluna = ['ENCONTRADO', 'NÃO ENCONTRADO']
    erroCol = []
    erro = []
    itens = 0
    erroNum = 0

    for i in abas_ATIVOS[c]['MÊS']:
        if i not in coluna:
            erroCol.append(i)
            erro.append(itens)
        itens += 1

    if len(erroCol) != 0:
        print(f'FORAM ENCONTRADOS {len(erroCol)} ERROS NA COLUNA "MÊS":')
        for i in erro:
            print('\033[91m', '\033[1m', '\033[4m', f'LINHA {i+3} >>> ERRO: {erroCol[erroNum]}', '\033[0m', '\033[0m', '\033[0m')
            erroNum += 1
            
            
            
    #ACHANDO ERROS NA COLUNA 'CONDIÇÃO SOCIAL'
    coluna = ["C.H.A.F", "C.H.D.F", "C.M.A.F", "C.M.D.F", "A.H.A.F", "A.H.D.F", "A.M.A.F", "A.M.D.F", "M.A", "M.I", "H.A", "H.I"]
    erroCol = []
    erro = []
    itens = 0
    erroNum = 0

    for i in abas_ATIVOS[c]['CONDIÇÃO SOCIAL']:
        if i not in coluna:
            erroCol.append(i)
            erro.append(itens)
        itens += 1

    if len(erroCol) != 0:
        print(f'FORAM ENCONTRADOS {len(erroCol)} ERROS NA COLUNA "CONDIÇÃO SOCIAL":')
        for i in erro:
            print('\033[91m', '\033[1m', '\033[4m', f'LINHA {i+3} >>> ERRO: {erroCol[erroNum]}', '\033[0m', '\033[0m', '\033[0m')
            erroNum += 1
            
            
            
    #ACHANDO ERROS NA COLUNA DE "DATA DA 1° ABORDAGEM"
    linha_DATA_1_ABORDAGEM = 0
    for row in abas_ATIVOS[c]['DATA 1° ABORDAGEM']:
        if type(row) == dt.datetime:
            if row < dt.datetime(2018, 1, 1, 0, 0):
                print(f"FORAM ENCONTRADOS ERROS NA COLUNA 'DATA DA 1ª ABORDAGEM':")
                print('\033[91m', '\033[1m', '\033[4m', f'LINHA {linha_DATA_1_ABORDAGEM+3} \nERRO: {row}\nO SEAS NÃO FUNCIONAVA ANTES DE 2018', '\033[0m', '\033[0m', '\033[0m')
        linha_DATA_1_ABORDAGEM += 1
        
        
# BLOCO DESATIVADO TEMPORARIAMENTE PQ A DANI PEDIU PARA TODOS RETIRAREM AS DATAS DE ÚLTIMA ABORDAGEM
#    #ACHANDO ERROS NA COLUNA DE "DATA DA ÚLTIMA ABORDAGEM"
#    erroCol = []
#    erro = []
#    itens = 0
#    erroNum = 0
#
#    for row in abas_ATIVOS[c]["DATA DA ÚLTIMA ABORDAGEM"]: #Aqui a gente tá pegando cada linha da aba de ativos, coluna mês
#        if type(row) == dt.datetime:#Se tiver fora do range aí bota na lista tbm
#            if row > dt.datetime.now() or row < dt.datetime(2018, 1, 1, 0, 0):
#                erro.append(itens)
#                erroCol.append(row)
#        elif type(row) == float:
#            erro.append(itens)
#            erroCol.append(row)
#        
#        itens += 1
#
#    if len(erroCol) != 0:
#        print(f'FORAM ENCONTRADOS {len(erroCol)} NA COLUNA "DATA DA ÚLTIMA ABORDAGEM"')
#        for i in erro:
#            print('\033[91m', '\033[1m', '\033[4m', f'LINHA {i+3} >>> ERRO: {erroCol[erroNum]}', '\033[0m', '\033[0m', '\033[0m')
#            erroNum += 1
    
    
        
    #ACHANDO ERROS NA COLUNA 'STATUS SEAS'
    coluna = ['ACOMPANHADO EQUIPAMENTO', 'ATENDIDO', 'IDENTIFICADO', 'ACOMPANHADO SEAS']
    erroCol = []
    erro = []
    itens = 0
    erroNum = 0

    for i in abas_ATIVOS[c]['STATUS SEAS']:
        if i not in coluna:
            erroCol.append(i)
            erro.append(itens)
        itens += 1

    if len(erroCol) != 0:
        print(f'FORAM ENCONTRADOS {len(erroCol)} ERROS NA COLUNA "STATUS SEAS":')
        for i in erro:
            print('\033[91m', '\033[1m', '\033[4m', f'LINHA {i+3} >>> ERRO: {erroCol[erroNum]}', '\033[0m', '\033[0m', '\033[0m')
            erroNum += 1
            
            
            
    #ACHANDO ERROS NA COLUNA 'NOME' --- NÃO TÁ FUNCIONANDO ---
    coluna = ['', ' ', 'Nan', 'nan']
    erroCol = []
    erro = []
    itens = 0

    for i in abas_ATIVOS[c]['NOME']:
        if i in coluna:
            erroCol.append(i)
            erro.append(itens)
        itens += 1

        
    
    #Encontrando o erro da coluna 'N° Form'
    #Se o total de itens do índice (Index) for diferente do valor do último item então falta um item no 'Nº FORM'
    #Obs: Se alguma planilha não tiver nenhum item em ativos vai dar um erro!
    #ISSO É MUITO IMPORTANTE PORQUE PODE DAR ERRO NAS OUTRAS LINHAS!
    if len(abas_ATIVOS[c].index) != abas_ATIVOS[c].index[-1]:
        toterros = abas_ATIVOS[c].index[-1] - abas_ATIVOS[c].index
        print(f'FORAM ENCONTRADOS ERROS NA COLUNA "Nº FORM"\n(LEMBRANDO QUE POR CAUSA DESSE ERRO, PODEM NÃO SER IDENTIFICADOS OUTROS ERROS NA TABELA!)')
        print('\033[91m', '\033[1m', '\033[4m',
              f'{len(abas_ATIVOS[c].index)} LINHAS PREENCHIDAS DE {abas_ATIVOS[c].index[-1]} LINHAS EXISTENTES!', '\033[0m', '\033[0m', '\033[0m', '\033[0m')
    print('-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=\n\n')
    c += 1



-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-= ENCONTRANDO ERROS NAS PLANILHAS =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-



In [15]:
print("\n-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-= CONCATENANDO TODOS OS DATAFRAMES =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-\n")
#Aqui a gente tá adicionando todos os arquivos lidos na lista "abas_ATIVOS" num mesmo dataframe
conct = pd.concat(abas_ATIVOS)

#Aqui a gente tá contando o total de gente encontrada no período!
c = 0
for linha in conct['MÊS']:
    if linha == 'ENCONTRADO':
        c += 1
#Aqui, se eu quiser colocar uma formatação legal, depois vou ter que adicionar cada item em uma lista e através dela formatar mais bonitinho.
print(f'ENCONTRADOS {c} no período')
print(f'NÃO ENCONTRADOS {conct[conct.columns[0]].count() - c} no período')
print(f'ATIVOS: {conct[conct.columns[0]].count()} registros no documento')
conct


-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-= CONCATENANDO TODOS OS DATAFRAMES =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-



ValueError: No objects to concatenate

In [16]:
print("\n-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-= FINALIZANDO E SALVANDO OS DADOS =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-\n")
#ANTES DISSO AQUI A GENTE PRECISA FAZER A PRIMEIRA COLUNA TER O NÚMERO DA EQUIPE E A REGIÃO ("EQUIPE 00 - REGIÃO ADMINISTRATIVA")
#ARRUMANDO A DATA PARA O NOME DO ARQUIVO
mes = {1: 'JANEIRO', 2: 'FEVEREIRO', 3:'MARÇO', 4: 'ABRIL', 5:'MAIO', 6:'JUNHO', 7:'JULHO', 8:'AGOSTO', 9:'SETEMBRO', 10:'OUTUBRO', 11:'NOVEMBRO', 12:'DEZEMBRO'}
#EXPORTANDO O DATAFRAME, COMO DOCUMENTO .CSV

dataMesCSV = dt.datetime.now().month #Posição da coluna do mês

if dataMesCSV == 1:
    dataMesCSV = mes[12]
else:
    dataMesCSV = mes[dataMesCSV - 1]

conct.to_excel(f"_01_Relatório Mensal - {dataMesCSV} DE {dt.datetime.now().year}.xlsx")
#print(conct)

tempoFinal = dt.datetime.now().minute
tempoExecusao = tempoFinal - tempoInicial
print(f'A execusão demorou {tempoExecusao} min.\nE já foi produzido o arquivo do excel!  ;)')


-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-= FINALIZANDO E SALVANDO OS DADOS =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-



NameError: name 'conct' is not defined